In [97]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets  import interact

# 1. 포인트클라우드 불러오기

In [56]:
folder_path = './용인금어'

In [59]:
file_list = os.listdir(folder_path)
file_list

['용인금어지구_group1_densified_point_cloud_part_1.xyz',
 '용인금어지구_group1_densified_point_cloud_part_2.xyz',
 '용인금어지구_group1_densified_point_cloud_part_3.xyz',
 '용인금어지구_group1_densified_point_cloud_part_4.xyz']

In [60]:
file_paths = [os.path.join(folder_path, i)for i in file_list]
file_paths

['./용인금어\\용인금어지구_group1_densified_point_cloud_part_1.xyz',
 './용인금어\\용인금어지구_group1_densified_point_cloud_part_2.xyz',
 './용인금어\\용인금어지구_group1_densified_point_cloud_part_3.xyz',
 './용인금어\\용인금어지구_group1_densified_point_cloud_part_4.xyz']

In [61]:
pcd_list = [o3d.io.read_point_cloud(i) for i in file_paths]

In [62]:
# 첫 번째 포인트클라우드 객체로 초기화
merged_pcd = pcd_list[0]

# 나머지 포인트클라우드 객체를 첫 번째 객체에 병합
for pcd in pcd_list[1:]:
    merged_pcd += pcd
pcd = merged_pcd

In [35]:
# pcd = o3d.io.read_point_cloud("./용인금어지구_group1_densified_point_cloud_part_1.xyz")
# # o3d.io.write_point_cloud("output.xyz", pcd)

In [63]:
print(pcd.points)

std::vector<Eigen::Vector3d> with 38601059 elements.
Use numpy.asarray() to access data.


# 2. 포인트 클라우드 가공

In [64]:
# Voxel 다운샘플링
downsampled_pcd = pcd.voxel_down_sample(voxel_size=1)

In [65]:
print(downsampled_pcd.points)

std::vector<Eigen::Vector3d> with 532511 elements.
Use numpy.asarray() to access data.


# 3. 필터링 알고리즘 함수(지형면 구분)

In [66]:
def filtering_points(pcd, grid, range_int):
    points = np.array(pcd.points)
    
    # 그리드 나누기
    grid_size = grid
    x_indices = np.floor(points[:,0]/grid_size).astype(int)
    y_indices = np.floor(points[:,1]/grid_size).astype(int)
    
    included_indices = []
    
    for x in np.unique(x_indices):
        for y in np.unique(y_indices):
            cell_indices = np.where((x_indices == x) & (y_indices == y))[0]
            if len(cell_indices) == 0:
                continue
            cell_points = points[cell_indices]
            z_min = np.min(cell_points[:, 2])
            z_max = np.max(cell_points[:, 2])
        
            range_z_min = z_min + range_int
        
            included_cell_indices = cell_indices[(cell_points[:, 2] <= range_z_min)]
        
            included_indices.extend(included_cell_indices)    

    # 제외된 포인트를 제외한 나머지 포인트 필터링
    remaining_pcd = pcd.select_by_index(included_indices)
    return remaining_pcd

In [92]:
new_pcd = filtering_points(downsampled_pcd,30,2)
new_pcd.points

std::vector<Eigen::Vector3d> with 168 elements.
Use numpy.asarray() to access data.

In [109]:
# 포인트 클라우드 제거 함수
grid = int(input('grid의 size: '))
range_int = int(input('높이 검토할 범위: '))

new_pcd = filtering_points(downsampled_pcd, grid, range_int)
o3d.visualization.draw_geometries([new_pcd], window_name="Included Point Cloud")

grid의 size:  30
높이 검토할 범위:  1


# 4. 포인트 클라우드 시각화

In [90]:
# 원본 포인트클라우드
o3d.visualization.draw_geometries([pcd], window_name="Original Point Cloud")

In [94]:
# 수정 포인트 클라우드
o3d.visualization.draw_geometries([new_pcd], window_name="Included Point Cloud")

In [46]:
# 시각화 객체 생성
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Original Point Cloud")
# 점군 데이터 추가
vis.add_geometry(pcd)
# 렌더링 옵션 가져오기
opt = vis.get_render_option()
# 점의 크기 설정
opt.point_size = 0.5  # 점 크기를 원하는 값으로 설정
# 시각화 실행
vis.run()
vis.destroy_window()

In [49]:
# 시각화 객체 생성
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Included Point Cloud")
# 점군 데이터 추가
vis.add_geometry(new_pcd)
# 렌더링 옵션 가져오기
opt = vis.get_render_option()
# 점의 크기 설정
opt.point_size = 1.0  # 점 크기를 원하는 값으로 설정
# 시각화 실행
vis.run()
vis.destroy_window()

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


# 99. test

In [ ]:
# z좌표의 통계 정보 출력
points = np.array(downsampled_pcd.points)
z_min = np.min(points[:,2])
z_max = np.max(points[:,2])

In [ ]:
# DBSCAN 클러스터링
eps = 1.5
min_points = 10
labels = np.array(downsampled_pcd.cluster_dbscan(eps=eps, min_points=min_points))
max_label = labels.max()
print(f"포인트 클라우드에서 {max_label + 1} 개의 클러스터가 검출됨")

In [ ]:
# 각 클러스터에 색상 지정
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # 노이즈 포인트는 검정색으로 설정
downsampled_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [ ]:
# 클러스터링 결과 시각화
o3d.visualization.draw_geometries([downsampled_pcd])

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
opt = vis.get_render_option()
opt.point_size = 0.5

vis.run()
vis.destroy_window()